In [3]:
!pip install langchain langchain-google-genai pypdf chromadb langchain_community langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [1]:
import os
from google.colab import userdata

try:
    # Use the key from Colab Secrets
    os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
except:
    # Fallback for direct input if you don't use Colab Secrets (less secure)
    print("Warning: 'GEMINI_API_KEY' not found in Colab Secrets. Prompting for manual input.")
    from getpass import getpass
    os.environ['GOOGLE_API_KEY'] = getpass("Enter your Gemini API Key: ")

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain # Moved from the failing cell
from langchain.memory import ConversationBufferMemory # Moved from the failing cell

# --- Configuration (unchanged) ---
# path of pdf
PDF_FILE_NAME = "/content/2024-wttc-introduction-to-ai.pdf"
LLM_MODEL = "gemini-2.5-flash"
EMBEDDING_MODEL = "models/embedding-001"
VECTOR_DB_PATH = "./chroma_colab_db"

def setup_rag_chain(file_path: str):
    # 1. Load Documents
    loader = PyPDFLoader(file_path)
    data = loader.load_and_split()

    # --- 2. Define Parent and Child Splitters ---
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)

    # --- 3. Setup Storage ---
    embeddings = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)
    vectorstore = Chroma.from_documents(data, embeddings)
    store = InMemoryStore()

    # --- 4. Create Parent Document Retriever ---
    retriever = ParentDocumentRetriever(
        vectorstore=vectorstore,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter,
    )
    retriever.add_documents(data)
    print("RAG chain setup complete using Parent Document Retriever.")

    # --- 5. Build the RAG Chain ---
    llm = ChatGoogleGenerativeAI(model=LLM_MODEL, temperature=0.2)
    prompt_template = """You are an expert Q&A assistant... (use the same strong prompt) ...
    Context: {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # --- 6. Setup Conversational Retrieval Chain (Moved from the failing cell) ---
    llm_with_history = ChatGoogleGenerativeAI(model=LLM_MODEL, temperature=0.2)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm_with_history,
        retriever=retriever, # Use the robust PDR you already built
        memory=memory
    )
    print("Conversational Retrieval Chain setup complete.")

    return rag_chain, qa_chain # Return both chains

# Execute the setup
rag_chain, qa_chain = setup_rag_chain(PDF_FILE_NAME)

RAG chain setup complete using Parent Document Retriever.
Conversational Retrieval Chain setup complete.


In [20]:
# Simple command-line loop for testing in Colab
print("-" * 50)
print("Welcome! Ask a question about your document.")
print("Type 'quit' or 'exit' to end the session.")
print("-" * 50)

while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Assistant: Goodbye!")
        break

    # Invoke the Conversational Retrieval Chain
    try:
        response = qa_chain.invoke({"question": user_input})
        print(f"Assistant: {response['answer']}")
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please check your API key and document setup.")

--------------------------------------------------
Welcome! Ask a question about your document.
Type 'quit' or 'exit' to end the session.
--------------------------------------------------
You: quit
Assistant: Goodbye!
